In [1]:
import pandas as pd
import numpy as np
import json
from typing import Optional, Sequence

from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
with open('data/delay_emails.json') as f:
    emails = json.load(f)

In [9]:
# Pydantic model for supplier delays
class ProductDelay(BaseModel):
    supplier: str
    qty: int
    product: str
    num_days_delay: int

class AllSupplierDelays(BaseModel):
    supplier_delay: Sequence[ProductDelay]

# Set up a Pydantic parser and prompt template
parser = PydanticOutputParser(pydantic_object=AllSupplierDelays)
prompt = PromptTemplate(
    template='''
        Answer the user query.
        \n{format_instructions}
        \n{query}
        \n",
    ''',
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Function to process each email and extract information
def process_email(email):
    # Initialize LLM
    llm = ChatOpenAI(model="gpt-4")
    chain = prompt | llm | parser
    result = chain.invoke({"query": email})
    
    # Convert Pydantic result to a dictionary
    supplier_delays_data = result.model_dump()

    # Flatten the nested structure for DataFrame creation
    data_dict = {'email': [], 'supplier': [], 'qty': [], 'product': [], 'num_days_delay': []}

    for record in supplier_delays_data['supplier_delay']:
        data_dict['email'].append(email)
        data_dict['supplier'].append(record['supplier'])
        data_dict['qty'].append(record['qty'])
        data_dict['product'].append(record['product'])
        data_dict['num_days_delay'].append(record['num_days_delay'])

    # Create a DataFrame from the flattened data
    delays_df = pd.DataFrame(data_dict)
    return delays_df

In [10]:
df_lst = []

for item in emails:
    email = item['email']
    df_lst.append(process_email(email))

In [11]:
product_delays = pd.concat(df_lst)
product_delays.to_csv("data/product_delays.csv", index=False)

In [33]:
product_delays = pd.read_csv("data/product_delays.csv")

In [16]:
products_df = pd.read_csv("data/products_data.csv")

In [34]:
product_delays.rename(columns={'product': 'product_name'},
                      inplace=True)

In [35]:
# TODO: implement logic to deal with plural and singular forms of the same product noun

# Hack: manually map product name for now
MANUAL_PRODUCT_MAP = {
    'Passionfruits': 'Passionfruit'
}

product_delays['product_name'] = product_delays['product_name'].map(MANUAL_PRODUCT_MAP).fillna(product_delays['product_name'])

In [40]:
merged_df = product_delays.merge(products_df, on=['supplier', 'product_name'])

In [42]:
# Assume for this grocery shop, we can place an urgent order with lead time of 1 day, for price = expedited price

# First check if days of inventory can cover lead time + delay time. If not, we need to order the delta days 1 day before delta days

merged_df['additional_order_days'] = merged_df['num_days_delay'] + merged_df['lead_time'] - merged_df['days_of_inventory']

In [56]:
merged_df['reorder_date'] = (pd.to_datetime('today') + pd.to_timedelta(merged_df['days_of_inventory'], unit='d') - pd.Timedelta(days=1)).dt.date

In [62]:
def check_price_reorder(row):
    '''
    Check if it is worthwhile to reorder or just go OOS
    logic: if expedited price is higher than product'shop price, 
    we will make a loss if we reorder, hence do not reorder
    '''
    if (row['expedited_price'] > row['price']) or (row['additional_order_days'] == 0):
        return np.nan
    return row['reorder_date']

def calculate_reorder_qty(row):
    '''
    Check if it is worthwhile to reorder or just go OOS
    logic: if expedited price is higher than product'shop price, 
    we will make a loss if we reorder, hence do not reorder
    '''
    if (row['expedited_price'] > row['price']) or (row['additional_order_days'] == 0):
        return np.nan
    return row['additional_order_days'] * row['daily_demand']

merged_df['reorder_date'] = merged_df.apply(check_price_reorder, axis=1)
merged_df['reorder_qty'] = merged_df.apply(calculate_reorder_qty, axis=1)

In [67]:
merged_df['value_of_items_delayed'] = (merged_df['price'] - merged_df['cogs']) * merged_df['daily_demand'] * merged_df['num_days_delay']

In [70]:
reorder_summary_df = merged_df[['product_id', 'product_name', 'price', 'expedited_price', 'daily_demand',
                                'lead_time', 'num_days_delay', 'days_of_inventory',	'additional_order_days',
                                'reorder_date', 'reorder_qty', 'value_of_items_delayed']]

reorder_summary_df.sort_values(by='value_of_items_delayed', ascending=False, inplace=True)

/var/folders/hj/c0k0z_096_n1727t5kbgcvxm0000gn/T/ipykernel_75081/3540302921.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reorder_summary_df.sort_values(by='value_of_items_delayed', ascending=False, inplace=True)


In [72]:
reorder_summary_df.to_csv("data/reorder_summary.csv", index=False)

In [79]:
merged_df.to_csv("data/product_delays_detailed.csv", index=False)